In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 读数据

In [67]:
data = pd.read_csv('air_data.csv',encoding='utf-8',parse_dates=['FFP_DATE','FIRST_FLIGHT_DATE'])
data['LOAD_TIME'] = pd.to_datetime(data['LOAD_TIME'])

In [68]:
data.head()

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006-11-02,2008-12-24,男,6,.,北京,CN,31.0,2014-03-31,...,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007-02-19,2007-08-03,男,6,NaN,北京,CN,42.0,2014-03-31,...,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007-02-01,2007-08-30,男,6,.,北京,CN,40.0,2014-03-31,...,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008-08-22,2008-08-23,男,5,Los Angeles,CA,US,64.0,2014-03-31,...,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009-04-10,2009-04-15,男,6,贵阳,贵州,CN,48.0,2014-03-31,...,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39


In [69]:
data.columns

Index(['MEMBER_NO', 'FFP_DATE', 'FIRST_FLIGHT_DATE', 'GENDER', 'FFP_TIER',
       'WORK_CITY', 'WORK_PROVINCE', 'WORK_COUNTRY', 'AGE', 'LOAD_TIME',
       'FLIGHT_COUNT', 'BP_SUM', 'EP_SUM_YR_1', 'EP_SUM_YR_2', 'SUM_YR_1',
       'SUM_YR_2', 'SEG_KM_SUM', 'WEIGHTED_SEG_KM', 'LAST_FLIGHT_DATE',
       'AVG_FLIGHT_COUNT', 'AVG_BP_SUM', 'BEGIN_TO_FIRST', 'LAST_TO_END',
       'AVG_INTERVAL', 'MAX_INTERVAL', 'ADD_POINTS_SUM_YR_1',
       'ADD_POINTS_SUM_YR_2', 'EXCHANGE_COUNT', 'avg_discount',
       'P1Y_Flight_Count', 'L1Y_Flight_Count', 'P1Y_BP_SUM', 'L1Y_BP_SUM',
       'EP_SUM', 'ADD_Point_SUM', 'Eli_Add_Point_Sum', 'L1Y_ELi_Add_Points',
       'Points_Sum', 'L1Y_Points_Sum', 'Ration_L1Y_Flight_Count',
       'Ration_P1Y_Flight_Count', 'Ration_P1Y_BPS', 'Ration_L1Y_BPS',
       'Point_NotFlight'],
      dtype='object')

# 数据预处理

## 删除票价为空的数据

In [70]:
# 先找到'SUM_YR_1'不为空的
# 再找到'SUM_YR_2'不为空的
# 找两个的交集x
# 信息提取
data = data[data['SUM_YR_1'].notnull() & data['SUM_YR_2'].notnull()]

In [71]:
data.shape

(62299, 44)

## 删除打折数据

- 丢弃票价为 0
- 平均折扣不为 0
- 总飞行公里数大于 0 的记录

In [73]:
# 丢弃 SUM_YR_1 和 SUM_YR_2 为0的数据
index1 = data['SUM_YR_1'] == 0
index2 = data['SUM_YR_2'] == 0
# 保留 avg_discount 为0的数据
index3 = data['avg_discount'] != 0
# 保留 SEG_KM_SUM 大于0的数据
index4 = data['SEG_KM_SUM'] > 0
data = data[-(index1 & index2 & index3 & index4)]
data.shape

(62051, 44)

# 特征构造

## L：会员入会时间距观测窗口结束的月数

In [76]:
# 筛选出'FFP_DATE'和'LOAD_TIME'的时间差，保留到月份
data['L'] = (data['LOAD_TIME'] - data['FFP_DATE'])/np.timedelta64(1,'M')

In [77]:
data.head()

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight,L
0,54993,2006-11-02,2008-12-24,男,6,.,北京,CN,31.0,2014-03-31,...,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50,88.905316
1,28065,2007-02-19,2007-08-03,男,6,NaN,北京,CN,42.0,2014-03-31,...,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33,85.324134
2,55106,2007-02-01,2007-08-30,男,6,.,北京,CN,40.0,2014-03-31,...,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26,85.915522
3,21189,2008-08-22,2008-08-23,男,5,Los Angeles,CA,US,64.0,2014-03-31,...,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12,67.253948
4,39546,2009-04-10,2009-04-15,男,6,贵阳,贵州,CN,48.0,2014-03-31,...,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39,59.664469


## R：客户最近一次乘坐公司飞机距观测窗口结束的月数

In [79]:
# 筛选出'LAST_TO_END'和'LOAD_TIME'的时间差，保留到月份
# 先将'LAST_TO_END'转换为时间格式
data['LAST_TO_END'] = pd.to_datetime(data['LAST_TO_END'],unit='D')
data['R'] = (data['LOAD_TIME'] - data['LAST_TO_END'])/np.timedelta64(1,'M')

In [80]:
data.head()

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight,L,R
0,54993,2006-11-02,2008-12-24,男,6,.,北京,CN,31.0,2014-03-31,...,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50,88.905316,530.902072
1,28065,2007-02-19,2007-08-03,男,6,NaN,北京,CN,42.0,2014-03-31,...,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33,85.324134,530.704943
2,55106,2007-02-01,2007-08-30,男,6,.,北京,CN,40.0,2014-03-31,...,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26,85.915522,530.573523
3,21189,2008-08-22,2008-08-23,男,5,Los Angeles,CA,US,64.0,2014-03-31,...,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12,67.253948,527.748003
4,39546,2009-04-10,2009-04-15,男,6,贵阳,贵州,CN,48.0,2014-03-31,...,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39,59.664469,530.770652


## F：客户在观测窗口内乘坐公司飞机的次数

## M：客户在观测窗口内累计的飞行里程数

## C：客户在观测窗口内乘坐舱位的平均折扣率

# 聚类

In [83]:
# 利用'L'进行聚类
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(data[['L']])
data['L_label'] = kmeans.labels_
data['L_label'].value_counts()

1    17991
3    13917
0    11895
4    10115
2     8133
Name: L_label, dtype: int64

In [84]:
# 利用'R'进行聚类
kmeans.fit(data[['R']])
data['R_label'] = kmeans.labels_
data['R_label'].value_counts()

0    26151
3    14361
2     9204
4     6833
1     5502
Name: R_label, dtype: int64